In [1]:
#-------------------------------------------------------------------------------------------------------------------------------
# IMPORT PACKAGES
#-------------------------------------------------------------------------------------------------------------------------------
%matplotlib inline

import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import math
import pandas as pd
from matplotlib.pyplot import cm
from tqdm import tnrange, tqdm_notebook
import seaborn as sns

In [2]:
#-------------------------------------------------------------------------------------------------------------------------------
# Functions
#-------------------------------------------------------------------------------------------------------------------------------
def moving_average(numbers, window_size):
    i = 0
    moving_averages = []
    while i < len(numbers) - window_size + 1:
        this_window = numbers[i : i + window_size]

        window_average = sum(this_window) / window_size
        moving_averages.append(window_average)
        i += 1
    return moving_averages

In [3]:
#-------------------------------------------------------------------------------------------------------------------------------
# LOGISTICS
#-------------------------------------------------------------------------------------------------------------------------------
# Define parts of file location
parent_directory = 'C:/Users/Michael Dawson/Documents/Github/Research'
program_name = "rl_loop"
# sub_directory = '2021-11-23--23-27-54'
# sub_directory = '2021-11-24--08-57-57'
# sub_directory = '2021-11-24--12-46-57'
# sub_directory = '2021-11-27--11-57-32'
# sub_directory = '2021-12-03--18-04-01'
# sub_directory = '2021-12-11--09-48-52'
# sub_directory = '2021-12-12--16-18-14'
# sub_directory = '2021-12-12--18-14-52'
# sub_directory = '2021-12-12--18-47-11'
# sub_directory = '2021-12-12--19-17-45'
# sub_directory = '2021-12-12--19-48-36'
# sub_directory = '2021-12-12--21-05-11'
# sub_directory = '2021-12-12--22-27-03' #good
# sub_directory = '2021-12-12--22-45-58'
# sub_directory = '2021-12-13--16-29-41' #verif
# sub_directory = '2021-12-13--17-51-24' #new train?
# sub_directory = '2021-12-13--18-38-18'
# sub_directory = '2021-12-13--19-42-12'
sub_directory = '2021-12-13--20-08-18' #actually good train
# sub_directory = '2021-12-13--20-23-38'
# sub_directory = '2021-12-13--20-39-02' #verify
# sub_directory = '2021-12-13--21-00-28' #incep verify


# Combine parts of file location
path = os.path.join(parent_directory,program_name,sub_directory)

# Read in test parameters
test_params = pd.read_csv(os.path.join(path,"test_params.csv"),index_col=0)

# Read in data from CSV into Pandas Dataframe
metadata = pd.read_csv(os.path.join(path,"metadata.csv"),index_col=0)

# Read in q table
q_table = pd.read_csv(os.path.join(path,"q_table.csv"),index_col=0)

# Read in data from testing spreadsheet
df = {}
for i in tnrange(0,len(metadata.index),leave=False):
    df[i] = pd.read_excel(os.path.join(path,"data.xlsx"),index_col=0,sheet_name="Test{}".format(i+1))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Michael Dawson/Documents/Github/Research/rl_loop/2021-12-13--20-08-18/test_params.csv'

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------------
# Settings
#-------------------------------------------------------------------------------------------------------------------------------
threshold = test_params['threshold'].tolist()[0]
moving_average_window = test_params['moving_average_window'].tolist()[0]
end_of_detections = test_params['stop'].tolist()[0]
start_temp = test_params['start_temp'].tolist()[0]
net = test_params['net'].tolist()[0]

if net==1: net='MobileNetV1-SSD'
if net==2: net='MobileNetV1-PPN'
if net==3: net='MobileNetV2-SSD'
if net==4: net='InceptionV2-SSD'

numberOfGoalAdjustmentVariants = len(pd.unique(metadata['all_la_records']))
numberOfInitialGoalVariants = len(pd.unique(metadata['all_ll_records']))

plt.style.use('default')
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 16
plt.rcParams['figure.dpi'] = 100

colors = cm.plasma(np.linspace(0, 1, len(metadata['all_la_records'])))

In [ ]:
# #-------------------------------------------------------------------------------------------------------------------------------
# # SUMMARY STATISTICS AND GRAPHS
# #-------------------------------------------------------------------------------------------------------------------------------
# # Plot all temps vs time in grid plot
# fig, axs = plt.subplots(nrows=numberOfInitialGoalVariants,ncols=numberOfGoalAdjustmentVariants, sharey='row', sharex='col')
# fig.set_figwidth(numberOfGoalAdjustmentVariants*8)
# fig.set_figheight(numberOfInitialGoalVariants*5)
# fig.set_dpi(200)
# for i in tnrange(0,numberOfInitialGoalVariants,desc='lg',leave=False) :
#     for j in tnrange(0, numberOfGoalAdjustmentVariants,desc='la',leave=False) :
#         iterator = j+i*numberOfGoalAdjustmentVariants        
#         color = colors[iterator]
#         working_df = df[iterator]
#         length = len(working_df['temps'])

#         graphing_times = np.linspace(working_df['times_loopLength'][0],
#                                      working_df['times_loopLength'][length-1],
#                                      int(working_df['times_loopLength'][length-1]))
#         graphing_times = graphing_times.tolist()
#         graphing_temp = []
#         graphing_temp.append(0)
#         for x in working_df['times_loopLength']:
#             for y in graphing_times:
#                 if x >= y and y > graphing_times[len(graphing_temp)-1]:
#                     graphing_times_index = graphing_times.index(y)
#                     temp_time_index = working_df.index[working_df['times_loopLength']==x].tolist()        
#                     graphing_temp.append(working_df['temps'][temp_time_index[0]])
                    
#         axs[i,j].step(working_df['times_temp'],
#                               working_df['temps'],
#                               label="CPU Temperature",
#                               color=color,
#                               alpha=0.5
#                              )
        
#         m_a_x = range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)+
#                       len(moving_average(graphing_temp,moving_average_window)))
#         m_a_y = moving_average(graphing_temp,moving_average_window)
#         axs[i,j].plot(m_a_x,m_a_y,     
#                                   color=color,
#                                   alpha=1,
#                                   label= '{} Second Average'.format(moving_average_window)
#                                  )
        
# #         axs[i,j].step(working_df['times_temp'],
# #                       working_df['temps2']/10,
# #                       label="GPU Temperature",
# #                       color='green',
# #                       alpha=0.5,
# #                       zorder=100
# #                      )

#         axs[i,j].legend(loc='lower right')
#         axs[i,j].set_ylim([40,90])

# #             for x in switch_record:
# #                 
# #                 if net_record[times_temp.index(x)] == 1 :
# #                     axs[0].axvline(x-0.2, color='r')
# #                     axs[0].text(x + 0.2,5,net_1_name,rotation=90)
# #                 if net_record[times_temp.index(x)] == 2 :
# #                     axs[0].axvline(x-0.2, color='c')
# #                     axs[0].text(x + 0.2,5,net_2_name,rotation=90)
# #                 if net_record[times_temp.index(x)] == 3 :
# #                     axs[0].axvline(x-0.2, color='m')
# #                     axs[0].text(x + 0.2,5,net_3_name,rotation=90)
# #                 if net_record[times_temp.index(x)] == 4 :
# #                     axs[0].axvline(x-0.2, color='y')
# #                     axs[0].text(x + 0.2,5,net_4_name,rotation=90)
                
#         axs[i,j].axhline(y=threshold, color='red',alpha=0.3)
#         axs[i,j].axhline(y=start_temp, color='red',alpha=0.3)
#         axs[i,j].axvline(end_of_detections, color='red',alpha=0.3)
# #         axs[i,j].text(end_of_detections+1,5,'End Detections',rotation=90,color='k')


#         if j == 0 : axs[i,j].set_ylabel('Temperature [$^\circ$C]' + " ; $lg$: " + 
#                                         str(metadata['all_ll_records'][iterator]))
#         if i == numberOfInitialGoalVariants-1: axs[i,j].set_xlabel('Time [s] ; $la$: {}'.format(
#                                                                    metadata['all_la_records'][iterator]))
        
#         axs[i,j].grid()

# # Save plot
# plot3_path = os.path.join(path, "All_Temps_net_{}.png".format(net))
# plt.savefig(plot3_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
# metadata['all_la_records'][iterator]

In [ ]:
#---------- Plot tri-plot for all tests ----------
# Temp
fig, axs = plt.subplots(nrows=3,ncols=1, sharey='row', sharex='col')
fig.set_figwidth(16)
fig.set_figheight(16)
temp = []
for i in tnrange(0,numberOfInitialGoalVariants,leave=False, desc='lg') :
    for j in tnrange(0, numberOfGoalAdjustmentVariants,leave=False, desc='GAC') :
        iterator=j+i*numberOfGoalAdjustmentVariants
        color = colors[iterator]

        working_df = df[iterator]
        length = len(working_df['temps'])

        graphing_times = np.linspace(working_df['times_loopLength'][0],
             working_df['times_loopLength'][length-1], 
             int(working_df['times_loopLength'][length-1]))
        graphing_times = graphing_times.tolist()
        graphing_temp = []
        graphing_temp.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_temp)-1]:
                    graphing_times_index = graphing_times.index(y)

                    temp_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_temp.append(working_df['temps'][temp_time_index[0]])
                    
        axs[0].step(working_df['times_temp'],working_df['temps'], 
                    label="Test {}".format(j+i*numberOfGoalAdjustmentVariants),
                    color=color,
                    alpha=0.5
                   )
        axs[0].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2) +
                          len(moving_average(graphing_temp,moving_average_window))),
                    moving_average(graphing_temp,moving_average_window),
                    label= "Test {}; {} Second Average".format(j+i*numberOfGoalAdjustmentVariants+1,moving_average_window),
                    color=color,
                    alpha=1)

        # Loops
        graphing_loopLength = []
        graphing_loopLength.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_loopLength)-1]:
                    graphing_times_index = graphing_times.index(y)
                    loopLength_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_loopLength.append(working_df['loopLength_record'][loopLength_time_index[0]])

        axs[1].step(working_df['times_loopLength'],
                    working_df['loopLength_record'], 
                    color=color, 
                    alpha=0.5
                   )
        axs[1].step(working_df['times_loopLength'],
                    working_df['loopLength_goal'], 
                    alpha=0.3,
                    color=color
                   )
        axs[1].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)+
                          len(moving_average(graphing_loopLength,moving_average_window))),
                    moving_average(graphing_loopLength,moving_average_window), 
                    color=color,
                    alpha=1
                   )   
        
        # CPU
        graphing_cpu = []
        graphing_cpu.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_cpu)-1]:
                    graphing_times_index = graphing_times.index(y)
                    cpu_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_cpu.append(working_df['cpu_record'][cpu_time_index[0]])

        axs[2].step(working_df['times_temp'],working_df['cpu_record'],color=color,alpha=0.5)
        axs[2].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)+
            len(moving_average(graphing_cpu,moving_average_window))),
            moving_average(graphing_cpu,moving_average_window), color=color, alpha=1)

        
        
#             for x in switch_record:
#                 
#                 if net_record[times_temp.index(x)] == 1 :
#                     axs[0].axvline(x-0.2, color='r')
#                     axs[0].text(x + 0.2,5,net_1_name,rotation=90)
#                 if net_record[times_temp.index(x)] == 2 :
#                     axs[0].axvline(x-0.2, color='c')
#                     axs[0].text(x + 0.2,5,net_2_name,rotation=90)
#                 if net_record[times_temp.index(x)] == 3 :
#                     axs[0].axvline(x-0.2, color='m')
#                     axs[0].text(x + 0.2,5,net_3_name,rotation=90)
#                 if net_record[times_temp.index(x)] == 4 :
#                     axs[0].axvline(x-0.2, color='y')
#                     axs[0].text(x + 0.2,5,net_4_name,rotation=90)

axs[0].set_ylim([40,90])
axs[0].axhline(y=start_temp, color='red',alpha=0.3)
axs[0].axhline(y=threshold, color='red',alpha=0.3)
axs[0].axvline(end_of_detections, color='red',alpha=0.3)
axs[0].set_ylabel('Temperature [$^\circ$C]')
axs[0].set_title('Temperature (Network: {})'.format(net))
axs[0].grid()

axs[1].set_ylim([0,5])        
axs[1].set_ylabel('Loop Length [s]')
axs[1].set_title('Loop Length (Network: {})'.format(net))
axs[1].grid()

axs[2].set_ylim([0,100])        
axs[2].set_ylabel('CPU Usage [%]')
axs[2].set_xlabel('Time [s]')
axs[2].set_title('CPU Usage (Network: {})'.format(net))
axs[2].grid()


fig.legend(loc='center left', bbox_to_anchor=(0.9, 0.5))
# axs[0].legend()

# Save plot
plot4_path = os.path.join(path, "Combined_color_net_{}.png".format(net))
plt.savefig(plot4_path,bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.scatter(df[0]['times_temp'].tolist(),df[0]['action_record'].tolist())
# fig = plt.figure(figsize=(10,6))
# sns.histplot(df[0]['action_record'].tolist())
# df[0]['action_record'].tolist()

In [ ]:
# fig = plt.figure(figsize=(10,6))
# sns.heatmap(qtable.pivot('Temperature','Loop Length',''))
# q_table

In [4]:
# q_table['']
vis_q_table = pd.DataFrame(q_table[['Temperature','Loop Length']])
for i in len(q_table)
    if q_table[['Linear Up','Linear Down','Linear','Nothing']].max(axis=1) == q_table[['Linear Up','Linear Down','Linear','Nothing']].min(axis=1)
    q_table = q_table.replace([])
vis_q_table['Action'] = q_table[['Linear Up','Linear Down','Linear','Nothing']].idxmax(axis=1)
vis_q_table = vis_q_table.replace(['Linear Up','Linear Down','Linear','Nothing'],[1,2,3,4])
# print(vis_q_table)
sns.heatmap(vis_q_table.pivot('Temperature','Loop Length','Action'))
policy_map_path = os.path.join(path, "policy_map.png")
plt.savefig(policy_map_path,bbox_inches="tight")

SyntaxError: invalid syntax (1873917840.py, line 3)

In [ ]:
mean_cpu = []
mean_loopLength_record = []
mean_temp = []
temp_error = []
first_temp = []
for i in tnrange(0,numberOfInitialGoalVariants,leave=False) :
    for j in tnrange(0, numberOfGoalAdjustmentVariants,leave=False) :
        iterator = j+i*numberOfGoalAdjustmentVariants
        working_df = df[iterator]
        mean_cpu.append(working_df['cpu_record'].mean())
        mean_loopLength_record.append(working_df['loopLength_record'].mean())
        mean_temp.append(working_df['temps'].mean())
        temp_error.append((working_df['temps'].sub(threshold)).pow(2).mean())
#         first_temp.append(working_df['temps'][0])
        
mean_record = {'mean_cpu':mean_cpu,
               'mean_loopLength_record':mean_loopLength_record,
               'mean_temp':mean_temp,
               'temp_error':temp_error,
#                'first_temp':first_temp
              }
mean_record_df = pd.DataFrame.from_dict(mean_record)
means_and_meta_df = metadata.join(mean_record_df)

In [ ]:
# sns.histplot(data=means_and_meta_df, x="first_temp", kde=True)

In [ ]:
# matplotlib.rcParams['font.size'] = 16

# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure(figsize=(6,6))
# ax = plt.axes(projection='3d')

# x = metadata['all_ll_records']
# y = metadata['all_la_records']
# z = means_and_meta_df['mean_temp']

# ax.scatter3D(x,y,z,c=z,cmap='plasma',linewidth=0.5)
# ax.set_xlabel('$lg$')
# ax.set_ylabel('$la$')
# ax.set_zlabel('Mean Temperature [$^\circ$C]')
# plt.xticks(np.linspace(0, max(x), numberOfInitialGoalVariants))
# plt.yticks(np.linspace(0, max(y), numberOfGoalAdjustmentVariants))

# plot5_path = os.path.join(path, "scatter_net_{}.png".format(net))
# plt.savefig(plot5_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
# fig = plt.figure(figsize=(6,6))

# ax = plt.axes(projection='3d')
# ax.plot_trisurf(x, y, z,cmap='plasma', edgecolor='none')
# ax.set_xlabel('$lg$')
# ax.set_ylabel('$la$')
# ax.set_zlabel('Mean Temperature [$^\circ$ C]')
# plt.xticks(np.linspace(0, max(x), numberOfInitialGoalVariants))
# plt.yticks(np.linspace(0, max(y), numberOfGoalAdjustmentVariants))
# ax.view_init(30, -30)


# plot6_path = os.path.join(path, "topology_net_{}.png".format(net))
# plt.savefig(plot6_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
index = means_and_meta_df['temp_error'].idxmin()

In [ ]:
# #---------- Plot tri-plot for all tests ----------

# # Temp
# fig, axs = plt.subplots(nrows=3,ncols=1, sharey='row', sharex='col')
# fig.set_figwidth(16)
# fig.set_figheight(16)
# temp = []
# for i in tnrange(0,numberOfInitialGoalVariants,leave=False) :
#     for j in tnrange(0, numberOfGoalAdjustmentVariants,leave=False) :
#         iterator = j+i*numberOfGoalAdjustmentVariants
#         zorder = iterator
        
#         if iterator == 10 :
#             color = 'blue'
#             zorder = 1000
#         else :
#             color = 'gray'

#         working_df = df[iterator]
#         length = len(working_df['temps'])

#         graphing_times = np.linspace(working_df['times_loopLength'][0],
#                                      working_df['times_loopLength'][length-1], 
#                                      int(working_df['times_loopLength'][length-1]))
#         graphing_times = graphing_times.tolist()
#         graphing_temp = []
#         graphing_temp.append(0)
#         for x in working_df['times_loopLength']:
#             for y in graphing_times:
#                 if x >= y and y > graphing_times[len(graphing_temp)-1]:
#                     graphing_times_index = graphing_times.index(y)

#                     temp_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
#                     graphing_temp.append(working_df['temps'][temp_time_index[0]])
                    
#         axs[0].step(working_df['times_temp'],working_df['temps'], 
#                     label="Test {} ; GAC: {} ; $lg$: {}".format(
#                         iterator+1,
#                         metadata['all_la_records'][iterator],
#                         metadata['all_ll_records'][iterator]),
#                     color=color,
#                     alpha=0.3,
#                     zorder=zorder
#                    )
#         axs[0].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)
#                     + len(moving_average(graphing_temp,moving_average_window))),
#                     moving_average(graphing_temp,moving_average_window),
#                     color=color,
#                     label= "Test {} ; {} Second Average".format(iterator+1,moving_average_window),
#                     alpha=1,
#                     zorder=zorder
#                    )

#         graphing_loopLength = []
#         graphing_loopLength.append(0)
#         for x in working_df['times_loopLength']:
#             for y in graphing_times:
#                 if x >= y and y > graphing_times[len(graphing_loopLength)-1]:
#                     graphing_times_index = graphing_times.index(y)
#                     loopLength_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
#                     graphing_loopLength.append(working_df['loopLength_record'][loopLength_time_index[0]])

#         axs[1].step(working_df['times_loopLength'], working_df['loopLength_record'],
#                     color=color, 
#                     alpha=0.5,
#                     zorder=zorder
#                    )
#         axs[1].step(working_df['times_loopLength'], working_df['loopLength_goal'], 
#                     alpha=1,
#                     color=color,
#                     zorder=zorder
#                    )
# #         axs[1].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)
# #                           +len(moving_average(graphing_loopLength,moving_average_window))),
# #                     moving_average(graphing_loopLength,moving_average_window), 
# #                     color=color,
# #                     zorder=zorder,
# #                     alpha=1
# #                    )

#         graphing_cpu = []
#         graphing_cpu.append(0)
#         for x in working_df['times_loopLength']:
#             for y in graphing_times:
#                 if x >= y and y > graphing_times[len(graphing_cpu)-1]:
#                     graphing_times_index = graphing_times.index(y)
#                     cpu_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
#                     graphing_cpu.append(working_df['cpu_record'][cpu_time_index[0]])

#         axs[2].step(working_df['times_temp'],working_df['cpu_record'],
#                     color=color,
#                     alpha=0.3,
#                     zorder=zorder
#                    )
#         axs[2].plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)+
#                     len(moving_average(graphing_cpu,moving_average_window))),
#                     moving_average(graphing_cpu,moving_average_window), 
#                     color=color, 
#                     alpha=1,
#                     zorder=zorder
#                    )


# axs[0].set_ylim([40,90])
# axs[0].axhline(y=start_temp, color='red',alpha=0.3)
# axs[0].axhline(y=threshold, color='red',alpha=0.3)
# axs[0].axvline(end_of_detections, color='red',alpha=0.3)
# axs[0].text(end_of_detections+1,5,'End Detections',rotation=90,color='k')
# axs[0].set_ylabel('Temperature [$^\circ$C]')
# axs[0].set_title('Temperature (Network: {})'.format(net))
# axs[0].grid()

# axs[1].set_ylim([0,3])        
# axs[1].set_ylabel('Loop Length [s]')
# axs[1].set_title('Loop Length (Network: {})'.format(net))
# axs[1].grid()

# axs[2].set_ylim([0,100])        
# axs[2].set_ylabel('CPU Usage [%]')
# axs[2].set_xlabel('Time [s]')
# axs[2].set_title('CPU Usage (Network: {})'.format(net))
# axs[2].grid()

# # fig.legend(loc='center left', bbox_to_anchor=(0.9, 0.5))

# # Save plot
# plot7_path = os.path.join(path, "Combined_gray_net_{}.png".format(net))
# plt.savefig(plot7_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
# from sklearn.decomposition import PCA
# X = pd.DataFrame()
# for i in tnrange(0,numberOfInitialGoalVariants,leave=False) :
#     for j in tnrange(0, numberOfGoalAdjustmentVariants,leave=False) :
#         working_df = df[iterator]
#         length = len(working_df['temps'])
#         q = pd.DataFrame(working_df.mean())        
#         X = X.append(q.T,ignore_index=True)
        
# from sklearn import preprocessing
# data_scaled = pd.DataFrame(preprocessing.scale(X),columns = X.columns) 
# pca = PCA(n_components=4)
# pca.fit_transform(X)

# # Dump components relations with features:
# # print(pd.DataFrame(pca.components_,columns=data_scaled.columns,index = ['PC-1','PC-2','PC-3','PC-4']))

# fig = plt.figure()
# PC_values = np.linspace(1,pca.n_components_,pca.n_components,endpoint=True)
# plt.plot(PC_values, pca.explained_variance_ratio_, 'ro-', linewidth=2)
# plt.title('Scree Plot (Network: {})'.format(net))
# plt.xlabel('Principal Component')
# plt.ylabel('Proportion of Variance Explained')

# plot8_path = os.path.join(path, "scree_net_{}.png".format(net))
# plt.savefig(plot8_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
# sns.heatmap(means_and_meta_df.corr())
# plot9_path = os.path.join(path, "mean_corr_heatmap_net_{}.png".format(net))
# plt.savefig(plot9_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
# means_and_meta_df['results'] = 10* (1/means_and_meta_df['mean_loopLength_record']) * 1/(means_and_meta_df["mean_cpu"]/100) 
# # means_and_meta_df

In [ ]:
# matplotlib.rcParams['font.size'] = 12

# pivoted = means_and_meta_df.pivot('all_la_records','all_ll_records','results')
# pivoted = pivoted.T
# axs = plt.axes()
# sns.heatmap(pivoted,
#             annot=True,
#             cmap='plasma',
#             fmt='.4g',
#             cbar_kws={'label': '(10x) FPS per fraction of CPU use ',"orientation": "horizontal"}
#            )
# axs.set_title('Image Processing Efficiency (Network: {})'.format(net))
# axs.set_ylabel('Initial Loop Length [s]')
# axs.set_xlabel('Loop Length Adjustment Coef. [s/$^\circ$C]')
# plot10_path = os.path.join(path, "results_net_{}.png".format(net))
# plt.savefig(plot10_path,bbox_inches="tight",dpi=400)
# plt.show()

In [ ]:
#---------- Plot tri-plot for all tests ----------
colors = cm.plasma(np.linspace(0, 1, 4))
# Temp
fig, axs = plt.subplots(nrows=1,ncols=1, sharey='row', sharex='col')
fig.set_figwidth(6)
fig.set_figheight(4)
temp = []
count = 0
for i in tnrange(0,1,leave=False) :
    for j in tnrange(0,1,leave=False) :
# for i in tnrange(0,1,leave=False) :
#     for j in tnrange(0,1,leave=False) :
        iterator = j+i*numberOfGoalAdjustmentVariants
        zorder = iterator
        color = colors[count]

        working_df = df[iterator]
        length = len(working_df['temps'])

        graphing_times = np.linspace(working_df['times_loopLength'][0],
                                     working_df['times_loopLength'][length-1], 
                                     int(working_df['times_loopLength'][length-1]))
        graphing_times = graphing_times.tolist()
        graphing_temp = []
        graphing_temp.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_temp)-1]:
                    graphing_times_index = graphing_times.index(y)

                    temp_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_temp.append(working_df['temps'][temp_time_index[0]])
                    
        axs.step(working_df['times_temp'],working_df['temps'], 
#                     label="Test {} ; $la$: {} ; $lg$: {}".format(
#                         iterator+1,
#                         metadata['all_la_records'][iterator],
#                         metadata['all_ll_records'][iterator]),
                    color=color,
                    alpha=0.3,
                    zorder=zorder
                   )
        axs.plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)
                    + len(moving_average(graphing_temp,moving_average_window))),
                    moving_average(graphing_temp,moving_average_window),
                    color=color,
#                     label= "Test {} ; {} Second Average".format(iterator+1,moving_average_window),
                    label="Test {}".format(
                        iterator+1),                    
                    alpha=1,
                    zorder=zorder
                   )
        count += 1

axs.set_ylim([50,90])
axs.axhline(y=threshold, color='red',alpha=0.3,label='Desired Temperature')
# axs.text(end_of_detections+1,5,'End Detections',rotation=90,color='k')
axs.set_ylabel('Temperature [$^\circ$C]')
axs.set_title('{} Temperature'.format(net))
axs.grid()
axs.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5))

axs.set_xlabel('Time [s]')



# Save plot
plot11_path = os.path.join(path, "TempEffectOfMetareasoning_net_{}.png".format(net))
plt.savefig(plot11_path,bbox_inches="tight",dpi=400)
plt.show()

In [ ]:
colors = cm.plasma(np.linspace(0, 1, 4))
fig, axs = plt.subplots(nrows=1,ncols=1, sharey='row', sharex='col')
fig.set_figwidth(6)
fig.set_figheight(4)
temp = []
count = 0
for i in tnrange(0,1,leave=False) :
    for j in tnrange(0,1,leave=False) :
        
        iterator = j+i*numberOfGoalAdjustmentVariants
        zorder = iterator
        color = colors[count]

        working_df = df[iterator]
        length = len(working_df['temps'])
        
        graphing_loopLength = []
        graphing_loopLength.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_loopLength)-1]:
                    graphing_times_index = graphing_times.index(y)
                    loopLength_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_loopLength.append(working_df['loopLength_record'][loopLength_time_index[0]])

        axs.step(working_df['times_loopLength'], working_df['loopLength_record'],
                    color=color, 
                    alpha=1,
                    zorder=zorder,
                    label="Test {}".format(
                        iterator+1),
                   )
        axs.step(working_df['times_loopLength'], working_df['pause_time_record'], 
                    alpha=0.5,
                    color=color,
                    zorder=zorder,
                    label= "Test {}; Pause Length".format(iterator+1),
                   )        
        count +=1
        
# axs[1].set_ylim([0,3])        
axs.set_ylabel('Loop Duration [s]')
axs.set_title('{} Loop Duration'.format(net))
axs.grid()
axs.set_xlabel('Time [s]')
axs.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5))

# Save plot
plot12_path = os.path.join(path, "LoopEffectOfMetareasoning_net_{}.png".format(net))
plt.savefig(plot12_path,bbox_inches="tight",dpi=400)
plt.show()

In [ ]:
colors = cm.plasma(np.linspace(0, 1, 4))
fig, axs = plt.subplots(nrows=1,ncols=1, sharey='row', sharex='col')
fig.set_figwidth(6)
fig.set_figheight(4)
temp = []
count = 0
for i in tnrange(0,1,leave=False) :
    for j in tnrange(0,1,leave=False) :
        iterator = j+i*numberOfGoalAdjustmentVariants
        zorder = iterator
        color = colors[count]

        working_df = df[iterator]
        length = len(working_df['temps'])
        
        graphing_cpu = []
        graphing_cpu.append(0)
        for x in working_df['times_loopLength']:
            for y in graphing_times:
                if x >= y and y > graphing_times[len(graphing_cpu)-1]:
                    graphing_times_index = graphing_times.index(y)
                    cpu_time_index = working_df.index[working_df['times_loopLength']==x].tolist()
                    graphing_cpu.append(working_df['cpu_record'][cpu_time_index[0]])

        axs.step(working_df['times_temp'],working_df['cpu_record'],
                    color=color,
                    alpha=0.3,
                    zorder=zorder,
#                     label="Test {} ; $la$: {} ; $lg$: {}".format(
#                         iterator+1,
#                         metadata['all_la_records'][iterator],
#                         metadata['all_ll_records'][iterator]),
                   )
        axs.plot(range(math.floor(moving_average_window/2),math.floor(moving_average_window/2)+
                    len(moving_average(graphing_cpu,moving_average_window))),
                    moving_average(graphing_cpu,moving_average_window), 
                    color=color, 
                    alpha=1,
                    zorder=zorder,
#                     label= "Test {} ; {} Second Average".format(iterator+1,moving_average_window),
                    label="Test {}".format(
                        iterator+1),                    
                   )
        count += 1
        
axs.set_ylim([0,100])        
axs.set_ylabel('CPU Usage [%]')
axs.set_xlabel('Time [s]')
axs.set_title('{} CPU Usage'.format(net))
axs.grid()
axs.legend(loc='lower center', bbox_to_anchor=(0.5, -0.8))

# Save plot
plot13_path = os.path.join(path, "CPUEffectOfMetareasoning_net_{}.png".format(net))
plt.savefig(plot13_path,bbox_inches="tight",dpi=400)
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# mask = means_and_meta_df['all_la_records']>0
X = means_and_meta_df[['all_la_records','all_ll_records']].to_numpy()
y = means_and_meta_df['temp_error'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4, random_state=3)

reg = LinearRegression().fit(X_train, y_train)

matplotlib.rcParams['font.size'] = 16
fig = plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
x = X_train[:,0]
y = X_train[:,1]
z = y_train
ax.scatter3D(x,y,z,c='blue',s=50,label='Train')
x = X_test[:,0]
y = X_test[:,1]
z = y_test
ax.scatter3D(x,y,z,c='red',s=50,label='Test')
# plt.xticks(np.linspace(0, max(x), numberOfInitialGoalVariants))
# plt.yticks(np.linspace(0, max(y), numberOfGoalAdjustmentVariants))


x = np.linspace(0,0.1,5)
y = np.linspace(0,2,5)
xx, yy = np.meshgrid(x,y)
X_working = np.hstack([xx.flatten().reshape(-1,1),yy.flatten().reshape(-1,1)])
z = reg.predict(X_working)
ax.scatter3D(xx,yy,z,label='Predict',color='green')

ax.set_ylabel('lg')
ax.set_xlabel('la')
ax.set_zlabel('Temperature Error [($^\circ$C)$^2$]')
plt.legend()
plt.show()

print("Train MSE: {}".format(mean_squared_error(y_train,reg.predict(X_train))))
print("Test MSE: {}".format(mean_squared_error(y_test,reg.predict(X_test))))

In [ ]:
from sklearn.linear_model import LinearRegression
df_mask=df[2]['times_temp']>=100
X = df[2][df_mask]['times_temp'].to_numpy().reshape(-1,1).astype(np.float32)
X_train = np.hstack([X,np.exp(-X)])
y = df[2][df_mask]['cpu_record'].to_numpy().astype(np.float32)
reg = LinearRegression().fit(X_train, y/100)
y_pred = reg.predict(X_train)

pause = df[2][df_mask]['pause_time_record'].to_numpy().reshape(-1,1).astype(np.float32)
# print(pause.flatten())
# from scipy.integrate import odeint
# def model(T,t):
#     k1 = 0.025
#     Tss = 80
#     Tamb = 22
#     dTdt = -k1 * (T - Tamb) + 1.55
# #     d2Tdt2 = 
#     return dTdt

# T0 = 50
# t = X.flatten()
# T = odeint(model,T0,t)

# from scipy.integrate import odeint
# def model(C,t):
#     k1 = 0.8
#     dCdt = -k1*(1/(t)) + 0.0025
#     return dCdt

# C0 = 0.9
# t = X.flatten()
# C= odeint(model,C0,t)

# np.exp(-k*pause[i])

# plt.plot(t,C,label='DEQ',c='green')
plt.scatter(X,y/100,label='Data',c='red')
plt.scatter(X,y_pred,label='LOBF')
plt.legend() 
print(reg.coef_)

In [ ]:
from sklearn.model_selection import train_test_split
df_mask=df[0]['times_temp']<300
X = df[0][df_mask]['times_temp'].to_numpy().reshape(-1,1).astype(np.float32)
y = df[0][df_mask]['temps'].to_numpy().reshape(-1,1).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.4, random_state=0)

# plt.scatter(X_train,y_train, label="Train", c='Blue', s=20, edgecolors='none')
# plt.scatter(X_test,y_test, label="Test", c='Red', s=20, edgecolors='none')

x = metadata['all_ll_records']
y = metadata['all_la_records']
z = means_and_meta_df['mean_temp']

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()  
        # Fully-Connected Layer: 1 (input data) -> 5 (hidden node)
        self.fc1 = nn.Linear(input_size, hidden_size)  
        
        # Non-Linear Layer
        self.sigmoid1 = nn.Sigmoid()
        # You can try other kinds as well
        # self.relu = nn.ReLU()
        # self.elu = nn.ELU()
        
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.sigmoid2 = nn.Sigmoid()
        
        # Fully-Connected Layer: 5 (hidden node) -> 1 (output)
        self.fc3 = nn.Linear(hidden_size, 1) 
        
    
    # Forward pass builds the model prediction from the inputs
    def forward(self, x):                              
        out = self.fc1(x)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.sigmoid2(out)
        out = self.fc3(out)
        return out

In [ ]:
# Build the network -- is it not trained yet
model = Net(input_size=1, hidden_size=4)

# What Loss function should we use? MSE!
criterion = nn.MSELoss()

# What Optimization procedure should we use?
learning_rate = 0.05
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)

In [ ]:
### Train the model

# Convert numpy arrays to torch tensors
inputs = torch.from_numpy(X_train)
targets = torch.from_numpy(y_train)

num_epochs = 5000
for epoch in range(num_epochs):

    ## Do Forward pass
    # Make predictions
    outputs = model(inputs)
    # Compute the loss function
    loss = criterion(outputs, targets)
    
    ## Update the model
    # Reset the optimizer gradients
    optimizer.zero_grad()
    # Compute the gradient of the loss function
    loss.backward()
    # Do an optimization step
    optimizer.step()
    
    # Print the loss
    if (epoch+1) % 200 == 0:
        print ('Epoch [{:4}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
# Plot the graph
plt.scatter(X_train, y_train, label='Data',s=10)
predicted = model(torch.from_numpy(np.linspace(0,300).reshape(-1,1).astype(np.float32))).detach().numpy()
plt.plot(np.linspace(0,300), predicted, label='Prediction',color='orange')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
a = np.arange(55,91,1)
b = np.linspace(0,5,51)
c = np.array(np.meshgrid(a, b)).T.reshape(-1,2)
d = np.zeros((5,1)).repeat(len(c),1).T


In [ ]:
b

In [ ]:
round(0.74,1)

In [ ]:
z = np.array((0,10,20,30,40,50,60))
print(z)

In [ ]:
y= int(np.where(z == max(z[2:5]))[0])
print(y)

In [ ]:
rand = np.random.uniform(2,6)
print(rand)
import pandas as pd

In [ ]:
possibleTemps = np.arange(45,91,1)
possibleLLs = np.around(np.linspace(0,5,51),1)
states = np.array(np.meshgrid(possibleTemps, possibleLLs)).T.reshape(-1,2)
print(states)
actions = np.zeros((4,1)).repeat(len(states),1).T
print(actions)
q_table = np.hstack((states,actions))
print(q_table)

q_table_df = pd.DataFrame(q_table, columns = ['Temperature','Loop Length','Linear','Boosted Linear','Quadratic','Nothing'])


In [ ]:
q_table_df.iloc[(258)].tolist()

In [ ]:
stateID = [50, 0.3]
q_row_index = q_table_df.index[(q_table_df['Temperature'] == 55.0) & (q_table_df['Loop Length'] == 0.3)].tolist()[0]
print(q_row_index)
q_row = q_table_df.iloc[[q_row_index]].to_numpy()[0]
print(q_row)
action = np.argwhere(q_row == np.max(q_row[2:]))[0][0] # 2, 3, 4, or 5 (because 0, 1 are state id)
print(action)

print(max(q_row[2:]))
print(sum(q_row[2:]))